In [0]:
%sql
USE CATALOG de_project_catalog;
USE SCHEMA olist_silver;

In [0]:
Bronze_base_path = "/Volumes/de_project_catalog/olist_bronze/raw_data/bronze"
Silver_base_path = "/Volumes/de_project_catalog/olist_silver/cleaned_data"

### Orders

In [0]:
bronze_orders_df = spark.read.parquet(
   f"{Bronze_base_path}/orders"
)

bronze_orders_df.printSchema()
bronze_orders_df.show(5)

In [0]:
corrupt_record_df  = bronze_orders_df.filter("corrupt_data is Not Null")
corrupt_record_df.display()